## Quickstart examples to understand the STL library 
this will be just a group of examples that will explain the basics of using the numpy-stl library.

Before starting to create STL files, we need to install all the needed libraries.

In [ ]:
!pip install numpy
!pip install numpy-stl
!pip install matplotlib
!pip install pillow

The first example here will create an empty STL file.

In [5]:
import numpy
from stl import mesh

# Creating a new mesh but make sure that the name is not "mesh".
# The vertice count is the number of triangles in the mesh.
VERTICE_COUNT = 100
# Create an array with the specified vertice count entries.
data = numpy.zeros(VERTICE_COUNT, dtype=mesh.Mesh.dtype)
# Generate the mesh using the data created.
TestMesh = mesh.Mesh(data, remove_empty_areas=False)
# The mesh normals (calculated automatically).
TestMesh.normals
# The mesh vectors of each triangle.
TestMesh.v0, TestMesh.v1, TestMesh.v2
# Accessing individual points (concatenation of v0, v1 and v2 in triplets).
assert (TestMesh.points[0][0:3] == TestMesh.v0[0]).all()
assert (TestMesh.points[0][3:6] == TestMesh.v1[0]).all()
assert (TestMesh.points[0][6:9] == TestMesh.v2[0]).all()
assert (TestMesh.points[1][0:3] == TestMesh.v0[1]).all()

TestMesh.save('NewSTLFile.stl')

#### Cube Generation :
This will generate a cube that is 100 mm wide (Play with the parameters to visualize the 3D changing that happened) and will later show a 3D visualization with PyPlot.

In [ ]:
import numpy as np
from stl import mesh
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import os

def GenerateCube(size=100.0):
    # Define the 8 vertices of the cube
    Vertices = np.array([
        [0, 0, 0],
        [size, 0, 0],
        [size, size, 0],
        [0, size, 0],
        [0, 0, size],
        [size, 0, size],
        [size, size, size],
        [0, size, size]
    ])

    # Define the 12 triangles composing the cube
    Faces = np.array([
        [0, 1, 3], [1, 2, 3],  # Bottom face
        [0, 1, 4], [1, 5, 4],  # Side face
        [1, 2, 5], [2, 6, 5],  # Side face
        [2, 3, 6], [3, 7, 6],  # Side face
        [0, 3, 4], [3, 7, 4],  # Side face
        [4, 5, 7], [5, 6, 7],  # Top face
    ])

    # Create the mesh
    CubeMesh = mesh.Mesh(np.zeros(Faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, face in enumerate(Faces):
        for j in range(3):
            CubeMesh.vectors[i][j] = Vertices[face[j], :]

    return CubeMesh

def VisualizeCube(CubeMesh):
    # Extract the triangles for visualization
    Figure = plt.figure()
    ax = Figure.add_subplot(111, projection='3d')

    # Create a list of triangles for Poly3DCollection
    triangles = [vector for vector in CubeMesh.vectors]

    # Create a 3D polygon collection for the cube faces
    poly3d = Poly3DCollection(triangles, alpha=0.7)
    poly3d.set_facecolor('lightblue')  # Set the color of the cube
    ax.add_collection3d(poly3d)

    # Extract cube's edges and draw them
    edges = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom edges
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top edges
        [0, 4], [1, 5], [2, 6], [3, 7],  # Side edges
    ]

    # Define the vertices of the cube
    size = CubeMesh.vectors.max()  # Assuming cube vertices range from 0 to size
    vertices = np.array([
        [0, 0, 0], [size, 0, 0], [size, size, 0], [0, size, 0],
        [0, 0, size], [size, 0, size], [size, size, size], [0, size, size]
    ])

    # Plot edges
    for edge in edges:
        edge_points = vertices[edge, :]
        ax.plot(edge_points[:, 0], edge_points[:, 1], edge_points[:, 2], color='black', linewidth=1)

    # Set the limits for better visualization
    ax.auto_scale_xyz([0, size], [0, size], [0, size])

    # Add labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('3D cube')

    plt.show()

try :
    os.remove("Cube.stl")
except FileNotFoundError :
    pass
# Generate a cube of size 1 and save to an STL file
Cube = GenerateCube(size=100.0)
Cube.save('Cube.stl')
VisualizeCube(Cube)

#### Creating multiple cubes :
Now moving onto generating multiple cubes and later combining them into one STL file.

In [ ]:
import numpy as np
from stl import mesh

def CreateCubeMesh(size=1.0, position=(0, 0, 0)):
    """Create a cube mesh with the given size and position."""
    x, y, z = position
    Vertices = np.array([
        [x, y, z], [x + size, y, z], [x + size, y + size, z], [x, y + size, z],  # Bottom
        [x, y, z + size], [x + size, y, z + size], [x + size, y + size, z + size], [x, y + size, z + size]  # Top
    ])
    Faces = np.array([
        [0, 1, 3], [1, 2, 3],  # Bottom face
        [0, 1, 4], [1, 5, 4],  # Side face
        [1, 2, 5], [2, 6, 5],  # Side face
        [2, 3, 6], [3, 7, 6],  # Side face
        [0, 3, 4], [3, 7, 4],  # Side face
        [4, 5, 7], [5, 6, 7],  # Top face
    ])
    Cube = mesh.Mesh(np.zeros(Faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, face in enumerate(Faces):
        for j in range(3):
            Cube.vectors[i][j] = Vertices[face[j], :]
    return Cube

def CombineMeshes(meshes):
    #Combine multiple meshes into one.
    combinedMesh = mesh.Mesh(np.concatenate([m.data for m in meshes]))
    return combinedMesh

"""
These are the cube parameters (Size and position), 
vary them to further understand how the generation 
and the positioning work.
"""
CubeSize = 10.0  # Size of each cube
Positions = [
    (0, 0, 0),  # First cube
    (8, 0, 0),  # Second cube, offset in X
    (0, 8, 0),  # Third cube, offset in Y
    (8, 8, 0)  # Fourth cube, offset in X and Y
]

# Generate the cubes
Cubes = [CreateCubeMesh(size=CubeSize, position=pos) for pos in Positions]

# Combine all cubes into one mesh
CombinedCubes = CombineMeshes(Cubes)

# Save the combined mesh to an STL file
CombinedCubes.save('MultipleCubes.stl')

print("Done !")

#### Analyzing an Image pixel by pixel :
The next step is to analyze each and every picture, convert it to BW and then generate cubes depending on the intensity ( in BW images, the R, G and B values are exact so we will need to extract the intensity of these pixels to later-on map the values to acquire a lithophane )

In [ ]:
from PIL import Image

# Load the image
ImagePath = "Cat.jpg"
Image = Image.open(ImagePath)

# Convert the image to grayscale
Image = Image.convert("L")

# Get the image dimensions
width, height = Image.size
print(f"Image Dimensions: {width}x{height}")

# Access pixels and analyze them
for y in range(height):  # Loop over rows
    for x in range(width):  # Loop over columns
        # Get the pixel value (grayscale intensity)
        Intensity = Image.getpixel((x, y))
        
        # Perform analysis (print pixel intensity)
        print(f"Pixel at ({x},{y}): Intensity={Intensity}")


## Lithophane Generation :
Now the last step is to generate a lithophane with the knowledge acquired from all the examples above which are in this order :<br />
• Read the image.<br />
• Convert it into grayscale (if it is not already black and white).<br />
• Analyze each pixel.<br />
• Create a cube that has the same base size, but the difference is in the height of this cube.<br />
• Add the cube to the STL file with the correct X and Y positions ( The Z position is always zero).<br />

In [ ]:
from PIL import Image
import numpy as np
from stl import mesh
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import os

##############
# We need a class that will be able to store the image info needed.
# Four info are needed : Pixel size, image width and image height
##############

class STLImage:
    STLName = "NoName"
    def __init__(self,GrayImage,ImageWidth,ImageHeight,PixelSize):
        self.GrayImage = GrayImage
        self.ImageWidth = ImageWidth
        self.ImageHeight = ImageHeight
        self.PixelSize = PixelSize
    def __str__(self):
        return f"FileName = {self.STLName} / Width*Height = {self.ImageWidth}*{self.ImageHeight} / PixelSize = {self.PixelSize}"
        



##############
# The First step is to analyze the image that is given into the input field and convert it accordingly
##############
def GetImageInfo(ImageDir,Rotation):
    # Transforming the image into grayscale to be able to analyze pixels.
    InpImage = Image.open(ImageDir).convert("L")
    # Get the image size
    Width,Height = InpImage.size
    if not Rotation :
        NewWidth = 100
        NewHeight = (Height/Width)*NewWidth
        PixelSize = NewWidth/Width
    else :
        NewHeight = 100
        NewWidth = (Height/Width)*NewHeight
        PixelSize = NewHeight/Height
    STLImage.STLName= ImageDir.split(".")[0] +".stl"
    GenImage = STLImage(InpImage,NewWidth,NewHeight,PixelSize)
    return GenImage

##############
# Generate a pixel (In this case a 3D pixel with a Z axis height dependent on the pixel intensity)
##############
def MakePixel(PixelSize, PixelIntensity, PixelPosition) :
    x, y = PixelPosition
    # This array will contain the points (corners of the cube which add up to 8 corners)
    vertices = np.array([
        # The cube(Pixel) bottom
        [x, y, 0],
        [x+PixelSize, y, 0],
        [x+PixelSize, y+PixelSize, 0],
        [x, y+PixelSize, 0],
        # The cube(Pixel) top
        [x, y, PixelIntensity],
        [x+PixelSize, y, PixelIntensity],
        [x+PixelSize, y+PixelSize, PixelIntensity],
        [x, y+PixelSize, PixelIntensity],
    ])
    # Each cube face is divided into two triangles, this will help to create the faces using the coordinates of three points.
    Faces = np.array([
        [0,1,3],[1,2,3],# Bottom
        [0,1,4],[1,5,4],# Side
        [1,2,5],[2,6,5],# Side
        [2,3,6],[3,7,6],# Side
        [0,3,4],[3,7,4],# Side
        [4,5,7],[5,6,7],# Top
    ])
    PixelMesh = mesh.Mesh(np.zeros(Faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, Face in enumerate(Faces):
        for j in range(3):
            PixelMesh.vectors[i][j] = vertices[Face[j], :]
    return PixelMesh


def MapIntensityToHeight(PixelIntensity, MinValue=0.5, MaxValue=3):
    MaxPixelIntensity = 255  # Maximum grayscale intensity
    return MaxValue - (PixelIntensity / MaxPixelIntensity) * (MaxValue - MinValue)

def CombineCubesIntoSTL(Cubes, OutputFileName="output.stl"):
    # Create an empty mesh to store all cube meshes
    CombinedMesh = mesh.Mesh(np.concatenate([cube.data for cube in Cubes]))
    # Save the combined mesh to an STL file
    CombinedMesh.save(OutputFileName)

Cubes=[]
GenImage = GetImageInfo("Cat.jpg",False)
print(GenImage)
print(GenImage.GrayImage.width)
for x in range(GenImage.GrayImage.width):
    for y in range(GenImage.GrayImage.height):
        PixelIntensity = MapIntensityToHeight(GenImage.GrayImage.getpixel((x, y)))  # Get pixel intensity
        Cubes.append(MakePixel(GenImage.PixelSize, PixelIntensity, (x * GenImage.PixelSize, y * GenImage.PixelSize)))
    print(f"The STL File is {(x/GenImage.GrayImage.width)*100:.2f}% complete")
print("The STL File is complete")
CombineCubesIntoSTL(Cubes, GenImage.STLName)